# 2D Metallic Flat Plate Magnetized in Exterior Field 
# FEM Solution of the Poisson Equation with Discontinuous Coefficient 
# using Ferrite.jl  

<b>Goal</b>: The goal of this notebook is to compute a numerical reference solution for the magnetization vector of metallic plate $\vec{M}(\vec{x})$ placed in homogeneous external magnetic field. This solution will be computed in the following two steps: 
1. solve the Poisson equation for the scalar magnetic potential $V_m(\vec{x})$;
2. compute the gradient of the potential times the magnetic susceptibility $\chi_m$ on the magnetic plate; 

Include figures here. 

## Next Steps 

<b>Post Processing</b> 
1. compute magnetic field $\vec{H}$ components and its magnitude. Write data to VTK file. See [Ferrite postprocessing](https://ferrite-fem.github.io/Ferrite.jl/stable/examples/postprocessing/); 
2. idem for magnetic flux $\vec{B}$;
3. idem for magnetization vector $\vec{M}$;

<b>Proper 2D Model Definition</b>
1. set geometry and material coefficients; 

<b> Extend from 2D to 3D</b>

## Import Packages

In [1]:
using Ferrite, SparseArrays

## Section 1: Test Case Description 
### Geometry 

The geometry consists of the following two elements: 
1. a plate represented as a cube of $10$ by $10$ by $0.02$ meters along the $x$, $y$ and $z$ axis. Origin in the middle of the plate; 
2. a surrounding box of air as a cube $20$ by $20$ by $0.04$ meters along the $x$, $y$ and $z$ axix; 

Remarks: 
1. Boundaries of the air box represent the far-field. Not sure whether box of air is large enough; 
2. Possibly take advance of symmetry in the field to reduce the computational domain to first octant of the coordinate axes. Use symmetry boundary conditions; 

### Mesh 

Generate the mesh. 

Remarks
1. Mesh allowed to be coarser in air than in plate domain; 

### Physics 

<b>Air</b> has exterior (unperturbed) magnetic field in $x$-direction given by $\vec{H}_0(\vec{x}) = (H_0,0,0)$ (units A/m). The scalar magnetic potential $V_m(\vec{x})$ for this field is given by $V_m(\vec{x}) = H_0 \, x$ (units A). Indeed, we have that by definition of the potential that $\vec{H}_0(\vec{x}) = \text{grad} \, V_m(\vec{x}) = \nabla V_m(\vec{x}) = (H_0,0,0)$. The potential $V_m(\vec{x}) = H_0 \, x$ will be applied as Dirichlet (fixedValue) boundary condition for the potential.   

<b>Magnetic plate</b> has magnetic susceptibility $\chi_m$, e.g., $\chi_m = 100$ (dimensionless). Plate placed in magnetic field will deform field locally (higher $\chi_m$ implies larger deformation of the exterior field due to metallic plate).  

The governing equations for stationary magnetic field are: 

$ \nabla \times \vec{H}(\vec{x}) = \vec{0}$ 

$ \nabla \cdot \vec{B}(\vec{x}) = 0 $

$\vec{B}(\vec{x}) = \mu_0 \, \vec{H}(\vec{x})$ (in air) 
and $\vec{B}(\vec{x}) = \mu_0 \, (1 + \chi_m) \, \vec{H}(\vec{x})$

<b>Differential Equation</b> 

The Poisson equation for the scalar magnetic potential $V_m(\vec{x})$ such that $\vec{H}(\vec{x}) = \nabla V_m(\vec{x})$ is given by 

$$
\nabla \cdot \left[ \mu_0 \, (1 + \chi_m) \, V_m(\vec{x}) \right] = 0 \text{ for } \vec{x} \in \Omega = \Omega_{air} \cup \Omega_{plate} 
$$

where $\chi_m = 0$ for $\vec{x} \in \Omega_{air}$ and $\chi_m = 100$ (dimensionless) for $\vec{x} \in \Omega_{plate}$.

<b>Boundary Conditions</b> 

As boundary condition on the scalar field $V_m(\vec{x})$ we impose that 
 
$$ V_m(\vec{x}) = H_0 \, x  \text{ for } \vec{x} \in \partial \Omega $$. 


### Solver

Using [Ferrite](https://ferrite-fem.github.io/Ferrite.jl/stable/examples/heat_equation/). Possibly consider [Gridap](https://gridap.github.io/Tutorials/dev/pages/t001_poisson/) as alternative in a later stage should the neeed arrise. 

### Post-Processing 

Compute the following quantities : 
1. the magnetic field $\vec{H} = \text{grad} [ u(\vec{x}) ]$ (the $x$, $y$, $z$ component and the magnitude) in the plate and air domain;
2. the magnetic flux $\vec{B} = \mu_0 \, \vec{H}$ (in air) and $\vec{B} = \mu_0 \, (1 + \chi_m) \, \vec{H}$; (as before)
3. the magnetization vector $\vec{M} = \chi_m \, \vec{H}$ in plate; (as before)

## Section 3: 3D Mockup of the Magnetized Plate  

In [2]:
# define spatially varying diffusion coefficient 
function my_diff_coeff(coord_qp)
    xbound = abs(coord_qp[1])<0.3
    ybound = abs(coord_qp[2])<0.3
    zbound = abs(coord_qp[3])<0.05    
    return 4*pi*1e-6*(1+99*(xbound*ybound*zbound))
end 

my_diff_coeff (generic function with 1 method)

In [3]:
# Ke: added spatially varying diffusion coefficient 
# fe: forces zero source term 
function assemble_element!(Ke::Matrix, fe::Vector, cellvalues::CellScalarValues, mycoords)
    n_basefuncs = getnbasefunctions(cellvalues)
    # Reset to 0
    fill!(Ke, 0)
    fill!(fe, 0)
    # Loop over quadrature points
    for q_point in 1:getnquadpoints(cellvalues)
        # Get the quadrature weight
        dΩ = getdetJdV(cellvalues, q_point)
        # ADDED: Get coord of quadrature point
        coords_qp = spatial_coordinate(cellvalues, q_point, mycoords)
        # ADDED: Evaluate spatially dependent diffusion coefficient in quad point 
        val_diff_coeff = my_diff_coeff(coords_qp)
        # Loop over test shape functions
        for i in 1:n_basefuncs
            δu  = shape_value(cellvalues, q_point, i)
            ∇δu = shape_gradient(cellvalues, q_point, i)
            # Add contribution to fe
            fe[i] += 0 * δu * dΩ
            # Loop over trial shape functions
            for j in 1:n_basefuncs
                ∇u = shape_gradient(cellvalues, q_point, j)
                # MODIFIED: Add contribution to Ke
                Ke[i, j] += val_diff_coeff * (∇δu ⋅ ∇u) * dΩ
            end
        end
    end
    return Ke, fe
end

assemble_element! (generic function with 1 method)

In [4]:
function assemble_global(cellvalues::CellScalarValues, K::SparseMatrixCSC, dh::DofHandler)
    # Allocate the element stiffness matrix and element force vector
    n_basefuncs = getnbasefunctions(cellvalues)
    Ke = zeros(n_basefuncs, n_basefuncs)
    fe = zeros(n_basefuncs)
    # Allocate global force vector f
    f = zeros(ndofs(dh))
    # Create an assembler
    assembler = start_assemble(K, f)
    # Loop over all cels
    for cell in CellIterator(dh)
        # Added: Get coordinates from current cell 
        coords = getcoordinates(cell)
        # Reinitialize cellvalues for this cell
        reinit!(cellvalues, cell)
        # Modified - Compute element contribution
        assemble_element!(Ke, fe, cellvalues, coords)
        # Assemble Ke and fe into K and f
        assemble!(assembler, celldofs(cell), Ke, fe)
    end
    return K, f
end

assemble_global (generic function with 1 method)

In [5]:
nels  = (100, 100, 10) # number of elements in each spatial direction
left  = Vec((-1.0, -1.0, -0.1)) # start point for geometry 
right = Vec((1.0, 1.0, 0.1))    # end point for geometry
grid  = generate_grid(Tetrahedron, nels, left, right)

Grid{3, Tetrahedron, Float64} with 600000 Tetrahedron cells and 112211 nodes

In [6]:
#grid

In [7]:
dim = 3
ip = Lagrange{dim, RefTetrahedron, 1}()
#ip = Lagrange{dim, RefCube, 1}()
qr = QuadratureRule{dim, RefTetrahedron}(1)
#qr = QuadratureRule{dim, RefCube}(1)
cellvalues = CellScalarValues(qr, ip);

In [8]:
dh = DofHandler(grid)
add!(dh, :u, 1)
close!(dh);

In [9]:
K = create_sparsity_pattern(dh);

In [10]:
ch = ConstraintHandler(dh);

In [11]:
grid

Grid{3, Tetrahedron, Float64} with 600000 Tetrahedron cells and 112211 nodes

In [12]:
# getfaceset(grid,"back")

In [13]:
∂Ω = union(
    getfaceset(grid, "left"),
    getfaceset(grid, "right"),
    getfaceset(grid, "front"),
    getfaceset(grid, "back"),
    getfaceset(grid, "top"),
    getfaceset(grid, "bottom"),
);

In [14]:
dbc = Dirichlet(:u, ∂Ω, (x, t) -> x[1])
add!(ch, dbc);

In [15]:
close!(ch)

ConstraintHandler:
  BCs:
    Field: u, Components: [1]

In [16]:
K, f = assemble_global(cellvalues, K, dh);

In [17]:
apply!(K, f, ch)
u = K \ f;

### Computing Fluxes

In [18]:
function compute_hfield(cellvalues::CellScalarValues{dim,T}, dh::DofHandler, a) where {dim,T}

    n = getnbasefunctions(cellvalues)
    cell_dofs = zeros(Int, n)
    nqp = getnquadpoints(cellvalues)

    # Allocate storage for the fluxes to store
    q = [Vec{3,T}[] for _ in 1:getncells(dh.grid)]

    for (cell_num, cell) in enumerate(CellIterator(dh))
        q_cell = q[cell_num]
        celldofs!(cell_dofs, dh, cell_num)
        aᵉ = a[cell_dofs]
        reinit!(cellvalues, cell)

        for q_point in 1:nqp
            q_qp = - function_gradient(cellvalues, q_point, aᵉ)
            push!(q_cell, q_qp)
        end
    end
    return q
end

function compute_bfield(cellvalues::CellScalarValues{dim,T}, dh::DofHandler, a) where {dim,T}

    n = getnbasefunctions(cellvalues)
    cell_dofs = zeros(Int, n)
    nqp = getnquadpoints(cellvalues)

    # Allocate storage for the fluxes to store
    q = [Vec{3,T}[] for _ in 1:getncells(dh.grid)]

    for (cell_num, cell) in enumerate(CellIterator(dh))
        q_cell = q[cell_num]
        celldofs!(cell_dofs, dh, cell_num)
        aᵉ = a[cell_dofs]
        reinit!(cellvalues, cell)
        coords = getcoordinates(cell)
        
        for q_point in 1:nqp
            coords_qp = spatial_coordinate(cellvalues, q_point, coords)
            val_diff_coeff = my_diff_coeff(coords_qp)
            q_qp = - val_diff_coeff*function_gradient(cellvalues, q_point, aᵉ)
            push!(q_cell, q_qp)
        end
    end
    return q
end

compute_bfield (generic function with 1 method)

In [19]:
h_gp = compute_hfield(cellvalues, dh, u);
b_gp = compute_bfield(cellvalues, dh, u);
#q_gp

In [20]:
projector = L2Projector(ip, grid);
#projector 

In [21]:
h_projected = project(projector, h_gp, qr; project_to_nodes=false);
b_projected = project(projector, b_gp, qr; project_to_nodes=false);
# q_projected

### Exporting to VTK 

In [22]:
vtk_grid("3d-uniform-magn_plate", dh) do vtk
    vtk_point_data(vtk, dh, u)
    vtk_point_data(vtk, projector, h_projected, "H")
    vtk_point_data(vtk, projector, b_projected, "B")
end

1-element Vector{String}:
 "3d-uniform-magn_plate.vtu"